In [ ]:
%load_ext autoreload
%autoreload 2

# session_handler
> A class to handle the session of the user.

In [ ]:
#| default_exp session_handler

In [ ]:
#| export
import requests
from fastcore.all import *
from terminal_llm.chat import Chat

In [ ]:
Path('.storage/chats').ls()[0].stem

'"Comparing Porsche and McLaren: Which Brand Offers the Best Weekend Fun for $50,000?"'

In [ ]:
'2' in Path('.storage/chats').ls()

False

In [ ]:
globtastic('.storage/chats', file_glob='2*')

(#1) ['.storage/chats/2. Comparing Porsche and McLaren: Which Brand Offers the Best Weekend Fun for $50,000?.pkl']

In [ ]:
globtastic('.storage/chats', file_glob='20*')

(#0) []

In [ ]:
load_pickle(globtastic('.storage/chats', file_glob='2*')[0])

[ChatMessage(role='user', content='Do you think Porches are better than McLarens?', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='I don\'t have personal experiences, preferences, or the ability to make judgments. Porches and McLarens serve different purposes in the automotive world. A Porsche is known for its sports cars and SUVs that offer a balance of performance and practicality. McLaren, on the other hand, is renowned for its high-performance supercars and racing vehicles. Neither is objectively "better" than the other, as it depends on individual needs, preferences, and budgets. Some people might prefer the everyday usability of a Porsche, while others might prioritize the extreme performance of a McLaren. It\'s essential to consider the specific models and features you\'re comparing when making such a judgment.', name=None, tool_calls=None),
 ChatMessage(role='user', content='I just would like to have fun on the weekends. Which brand would be the best for t

In [ ]:
for f in Path('.storage/chats').ls(): print(f.stem)

"Comparing Porsche and McLaren: Which Brand Offers the Best Weekend Fun for $50,000?"
"Exploring the Meaning and Purpose of Life: A Personal and Subjective Inquiry"


In [ ]:
?enumerate

Init signature: enumerate(iterable, start=0)
Docstring:     
Return an enumerate object.

  iterable
    an object supporting iteration

The enumerate object yields pairs containing a count (from start, which
defaults to zero) and a value yielded by the iterable argument.

enumerate is useful for obtaining an indexed list:
    (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
Type:           type
Subclasses:     

In [ ]:
for n, f in enumerate(Path('.storage/chats').ls(), 1): print(f'{n}: {f.stem}')

1: "Comparing Porsche and McLaren: Which Brand Offers the Best Weekend Fun for $50,000?"
2: "Exploring the Meaning and Purpose of Life: A Personal and Subjective Inquiry"


In [ ]:
#| export
class SessionHandler():
  '''Handles application session by managing user input and output.'''
  def __init__(self, interface):
    '''Initializes session handler.'''
    store_attr()
    self.is_running = True
  
  def start_app(self):
    '''Starts the application. If storage directory exists, reads API key and model. Otherwise, prompts user to set API key and model.'''
    if Path('.storage').exists():
      self.read_api_key()
      self.read_model()
    else:
      Path('.storage/chats').mkdir(parents=True)
      self.set_api_key()
      self.set_model()
  
  def process_menu(self, choice):
    '''Processes menu choice.'''
    if   choice == '0': self.is_running = False
    elif choice == '1': self.interface.settings()
    elif choice == '2': self.start_chat()
    elif choice == '3': self.interface.previous_chats()
    else:               print("Invalid choice.")
  
  def process_settings(self, choice):
    '''Processes settings choice.'''
    if   choice == '0': pass
    elif choice == '1': self.set_api_key()
    elif choice == '2': self.set_model()
    else: 
      print('Invalid choice.')
      self.interface.settings()
  
  def process_previous_chats(self, choice):
    '''Processes previous chats choice. If chat is found, resumes chat. Otherwise, prompts user to select another chat.'''
    if choice == '0': pass
    else:
      result = globtastic('.storage/chats', file_glob=f'{choice}. *')
      if result: 
        name = Path(result[0]).stem
        print(f'Resuming {name}.')
        self.start_chat(load_pickle(result[0]), chat_name=name)
      else: 
        print('Chat not found.')
        self.interface.previous_chats()
  
  def start_chat(self, history=[], chat_name=None):
    '''Starts chat session.'''
    print('Chat started. Type \\exit to end chat.')
    self.chat = Chat(self.api_key, self.model, history)
    while True:
      user_input = input('\nYou: ')
      if user_input.strip() == '\\exit':
        if chat_name:
          save_pickle(f'.storage/chats/{chat_name}.pkl', self.chat.history)
        else:
          num_chat = len(Path('.storage/chats').ls()) + 1
          # History from prior chats keeps persisting for some reason.
          # chat_name = self.chat('Provide a concise title for this conversation.', save_history=False).strip('"')
          chat_name = input('Name this conversation: ').strip()
          save_pickle(f'.storage/chats/{num_chat}. {chat_name}.pkl', self.chat.history)
        break
      else: 
        response = self.chat(user_input)
        print(f'Assistant: {response}')
    
  def read_model(self):
    '''Reads model from file. If file not found, prompts user to set model.'''
    try: self.model = self.read_file('.storage/model.txt')
    except FileNotFoundError: 
      print('Model not found.')
      self.set_model()

  def read_api_key(self):
    '''Reads API key from file. If file not found, prompts user to set API key.'''
    try: self.api_key = self.read_file('.storage/api_key.txt')
    except FileNotFoundError: 
      print("API key not found.")
      self.set_api_key()

  def set_model(self):
    '''Prompts user to set model and saves it to file.'''
    self.model = input('Enter model: ').strip()
    while self.model not in ['open-mistral-7b', 'open-mixtral-8x7b', 'mistral-small-latest', 'mistral-medium-latest', 'mistral-large-latest']: self.model = input('Invalid model. Enter model: ')
    self.write_file('.storage/model.txt', self.model)
    print('Model saved.')

  def set_api_key(self):
    '''Prompts user to set API key and saves it to file.'''
    self.api_key = input('Enter API Key: ')
    while not self.check_api_key(): self.api_key = input('Invalid API key. Enter API Key: ')
    self.write_file('.storage/api_key.txt', self.api_key)
    print('API key saved.')

  def read_file(self, f_path):
    '''Reads file and returns content.'''
    with open(f_path, 'r') as f: return f.read()
  
  def write_file(self, f_path, content):
    '''Writes content to file.'''
    with open(f_path, 'w') as f: f.write(content)
  
  def check_api_key(self):
    '''Checks if API key is valid.'''
    h = {'Content-Type': 'application/json', 'Accept': 'application/json', 'Authorization': f'Bearer {self.api_key}'}
    d = {'model': 'open-mistral-7b', 'messages': [{'role': 'user', 'content': 'THIS IS AN API KEY TEST!'}]}
    r = requests.post('https://api.mistral.ai/v1/chat/completions', headers=h, json=d)
    return True if r.status_code == 200 else False

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
show_doc(SessionHandler)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L12){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler

>      SessionHandler (interface)

Handles application session by managing user input and output.

In [ ]:
show_doc(SessionHandler.start_app)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L17){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.start_app

>      SessionHandler.start_app ()

Starts the application. If storage directory exists, reads API key and model. Otherwise, prompts user to set API key and model.

In [ ]:
show_doc(SessionHandler.process_menu)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L26){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.process_menu

>      SessionHandler.process_menu (choice)

Processes menu choice.

In [ ]:
show_doc(SessionHandler.process_settings)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L33){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.process_settings

>      SessionHandler.process_settings (choice)

Processes settings choice.

In [ ]:
show_doc(SessionHandler.start_chat)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L41){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.start_chat

>      SessionHandler.start_chat ()

Starts chat session.

In [ ]:
show_doc(SessionHandler.read_model)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L52){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.read_model

>      SessionHandler.read_model ()

Reads model from file. If file not found, prompts user to set model.

In [ ]:
show_doc(SessionHandler.read_api_key)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L58){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.read_api_key

>      SessionHandler.read_api_key ()

Reads API key from file. If file not found, prompts user to set API key.

In [ ]:
show_doc(SessionHandler.set_model)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L64){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.set_model

>      SessionHandler.set_model ()

Prompts user to set model and saves it to file.

In [ ]:
show_doc(SessionHandler.set_api_key)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L70){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.set_api_key

>      SessionHandler.set_api_key ()

Prompts user to set API key and saves it to file.

In [ ]:
show_doc(SessionHandler.read_file)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L76){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.read_file

>      SessionHandler.read_file (f_path)

Reads file and returns content.

In [ ]:
show_doc(SessionHandler.write_file)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L79){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.write_file

>      SessionHandler.write_file (f_path, content)

Writes content to file.

In [ ]:
show_doc(SessionHandler.check_api_key)

---

[source](https://github.com/ForBo7/terminal_llm/blob/main/terminal_llm/session_handler.py#L82){target="_blank" style="float:right; font-size:smaller"}

### SessionHandler.check_api_key

>      SessionHandler.check_api_key ()

Checks if API key is valid.

In [ ]:
show_doc(SessionHandler.process_previous_chats)

---

### SessionHandler.process_previous_chats

>      SessionHandler.process_previous_chats (choice)

Processes previous chats choice. If chat is found, resumes chat. Otherwise, prompts user to select another chat.

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()